In [177]:
import numpy as np
import torch
import torch.optim as optim

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
from torchvision import transforms
import torch.utils.data

from timeit import default_timer as timer
import random
from random import shuffle

import matplotlib.pyplot as plt  
%matplotlib inline

from PIL import Image

device = torch.device("cuda:0")

In [178]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [179]:
trainset = torchvision.datasets.ImageFolder("../data/Training-Dataset/Resize-Augmented-Masks/", transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60, shuffle=True)
testset = torchvision.datasets.ImageFolder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/", transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=60, shuffle=True)

In [180]:
iter_ = iter(train_data_loader)
bimg, blabel = next(iter_)

print('Batch Label shape: ', blabel.shape)
print('The Batched tensors return a collection of {} RGB images ({} channel, {} height pixels, {} width pixels)'.format(bimg.shape[0],
                                                                                                                        bimg.shape[1],
                                                                                                                        bimg.shape[2],
                                                                                                                        bimg.shape[3]))
print('In the case of the labels, we obtain {} batched integers, one per image'.format(blabel.shape[0]))

Batch Label shape:  torch.Size([15])
The Batched tensors return a collection of 15 RGB images (1 channel, 256 height pixels, 256 width pixels)
In the case of the labels, we obtain 15 batched integers, one per image


In [181]:
class ConvBlock(nn.Module):

    def __init__(self, num_inp_channels, num_out_fmaps, kernel_size, stride=1):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.conv = nn.Conv2d(num_inp_channels, num_out_fmaps, kernel_size, stride=stride)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        P_ = self.kernel_size // 2
        if self.stride > 1:
          P = (P_ - 1, P_, P_  - 1, P_ )
        else:
          P = (P_, P_, P_, P_)
        x = self.conv(F.pad(x, P, mode='constant'))
        return self.relu(x)

In [182]:
class BigNet(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.conv1 = ConvBlock(1, 16, 2, stride=2)
    self.conv2 = ConvBlock(16, 32, 2, stride=2)
    self.conv3 = ConvBlock(32, 64, 2, stride=2)
    self.conv4 = ConvBlock(64, 128, 2, stride=2)
    
    self.mlp = nn.Sequential(
        nn.Linear(128 * 16 * 16, 512),
        nn.ReLU(inplace=True),
        nn.Linear(512, 5),
        nn.LogSoftmax(dim=-1)
    )
    
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    bsz, nch, height, width = x.shape
    x = x.view(bsz, -1)
    #print(bsz, nch, height, width)
    y = self.mlp(x)
    return y

In [183]:
def model_params(model):
    # from: https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [184]:
cnn = BigNet()
model_params(cnn)

16823605

In [185]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.1)
epochs = 20
steps = 0
running_loss = 0
train_losses = []
test_losses = []
print_every = 10

In [197]:
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        #inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = cnn.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            cnn.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    #print(labels[0])
                    #inputs, labels = inputs.to(device), labels.to(device)
                    logps = cnn.forward(inputs)
                    
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            cnn.train()

#torch.save(model, 'aerialmodel.pth')

tensor([[0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],


tensor([[0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],
        [0.2147, 0.1989, 0.1988, 0.1955, 0.1921],


KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()